In [40]:
import config
from pathlib import Path
from typing import List, Iterator
import os
import pandas as pd

import pickle
from wmutils.file import iterate_through_files_in_nested_folders
from wmutils.multithreading import parallelize_tasks

import util
import imports_parser

In [41]:
notebook_directory = config.path_default.joinpath("notebooks")
dataframe_path = config.path_default.joinpath("nberror_g_all.xlsx")
dataframe_path = config.path_default.joinpath("nberror_k.xlsx")

In [42]:
df = pd.read_excel(dataframe_path, header=0)
print(f"{df.columns=}, {len(df)}")

df.columns=Index(['fname', 'ename', 'evalue', 'traceback'], dtype='object'), 6776


In [43]:
from wmutils.file import iterate_through_files_in_nested_folders

iterator = iterate_through_files_in_nested_folders(notebook_directory, 10_000)
notebooks = set(os.path.basename(file) for file in iterator)

print(len(notebooks))

def has_notebook_file(notebook_name: str) -> bool:
    return notebook_name in notebooks


mask = df["fname"].apply(has_notebook_file)
df = df[mask]

del notebooks
del iterator

print(f"{df.columns=}, {len(df)}")

4365
df.columns=Index(['fname', 'ename', 'evalue', 'traceback'], dtype='object'), 6776


In [44]:
# df = df.sample(1000)

## Method 1

In [45]:
df["lib"] = df["traceback"].apply(util.extract_lib)

null_count = 100 * (len(df) - len(df[df["lib"].isnull()])) / len(df)
print(f"Extracted {null_count:.2f}% of the underlying packages using simple method.")

Extracted 51.95% of the underlying packages using simple method.


## Method 2

In [46]:
# Applies method 2
pickle_path = config.path_default.joinpath("lib_classes.pickle")
if not os.path.exists(pickle_path):
    library_classes = util.export_classes_from_modules(
        config.top_lib_names, pickle_path
    )
else:
    with open(pickle_path, "rb") as pickle_file:
        library_classes = pickle.load(pickle_file)

print("Loaded pickled class data.")

Loaded pickled class data.


In [47]:
def __get_imports(notebook: str) -> str:
    """Returns the imports of the specified notebook."""
    notebook_path = notebook_directory.joinpath(notebook)
    imports = imports_parser.get_imports_nbs_static(
        notebook_path, get_imports_func=imports_parser.get_imports_line_outermost
    )

    import_str = ",".join(imports)
    return import_str


nb_imports = df[["fname"]].copy()
nb_imports["imports"] = nb_imports["fname"].apply(__get_imports)

print("Loaded nb imports.")

Loaded nb imports.


In [48]:
q = nb_imports[nb_imports['imports'] != ""]

q

,fname,imports


In [49]:
kwargs = {
    "lib_names": config.top_lib_names,
    "df_imports": nb_imports,
    "lib_classes_dict": library_classes,
}
df["lib2"] = df.apply(util.extract_lib_2, axis=1, **kwargs)

null_count = 100 * (len(df) - len(df[df["lib2"].isnull()])) / len(df)
print(
    f"Extracted {null_count:.2f}% of the underlying packages using the less simple method."
)

AttributeError: 'DataFrame' object has no attribute 'lib_alias'